In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv() 

llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
llm.invoke("Pode me dizer qual o prédio mais alto do mundo?")

AIMessage(content='O prédio mais alto do mundo, até a minha última atualização em outubro de 2023, é o Burj Khalifa, localizado em Dubai, Emirados Árabes Unidos. Ele tem uma altura de 828 metros (2.717 pés) e foi inaugurado em 2010. Se precisar de informações mais recentes ou detalhes sobre outros arranha-céus, fique à vontade para perguntar!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 82, 'prompt_tokens': 18, 'total_tokens': 100, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'stop', 'logprobs': None}, id='run-5f62c987-1d46-490c-b0f5-d5ac81d131f7-0', usage_metadata={'input_tokens': 18, 'output_tokens': 82, 'total_tokens': 100, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'out

In [2]:
from langchain_core.output_parsers import StrOutputParser
llm_com_parser = llm | StrOutputParser()
llm_com_parser.invoke("Pode me dizer qual o prédio mais alto do mundo?")

'O prédio mais alto do mundo, até a minha data de corte em outubro de 2023, é o Burj Khalifa, localizado em Dubai, nos Emirados Árabes Unidos. O Burj Khalifa tem uma altura de 828 metros (2.717 pés) e foi inaugurado em 2010. Ele é conhecido por sua arquitetura impressionante e por ser um marco icônico da cidade.'

In [3]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate

from pydantic import BaseModel, Field

class Pessoa(BaseModel):
    nome: str = Field(description="nome da pessoa")
    idade: int = Field(description="idade da pessoa")
    hobby: str = Field(description="algo que a pessoa goste")

parser_pydantic = PydanticOutputParser(pydantic_object=Pessoa)

prompt = PromptTemplate(
    template="Formate adequadamente o texto enviado.\n{format_instructions}\n{texto}\n",
    input_variables=["texto"],
    partial_variables={"format_instructions": parser_pydantic.get_format_instructions()},
)

chain = prompt | llm | parser_pydantic
output = chain.invoke({"texto": "Paulo é um homem de 22 anos que gosta de basquete"})
output


Pessoa(nome='Paulo', idade=22, hobby='basquete')

In [4]:
from typing import List

class Lista(BaseModel):
    """Lista de informações sobre pessoas"""

    people: List[Pessoa]   

In [ ]:
parser_pydantic = PydanticOutputParser(pydantic_object=Lista)

prompt = PromptTemplate(
    template="Formate adequadamente o texto enviado.\n{format_instructions}\n{texto}\n",
    input_variables=["texto"],
    partial_variables={"format_instructions": parser_pydantic.get_format_instructions()},
)

# And a query intended to prompt a language model to populate the data structure.
chain = prompt | llm | parser_pydantic
output = chain.invoke({"texto": "Paulo é tem 22 anos, gosta de basquete e a Carla é tem 31 anos e gosta de Formula 1"})
output

Lista(people=[Pessoa(nome='Paulo', idade=22, hobby='basquete'), Pessoa(nome='Carla', idade=31, hobby='Formula 1')])

In [5]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

In [6]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

chat_history = ChatMessageHistory()

chain_memoria = RunnableWithMessageHistory(
    prompt | llm,
    lambda session_id: chat_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [ ]:
chain_memoria.invoke(
    {"input": "Olá"},
    {"configurable": {"session_id": "unused"}},
)

AIMessage(content='Olá!  Como posso ajudar?\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-e5004cff-8c24-4615-ad17-ac677054b2a1-0', usage_metadata={'input_tokens': 18, 'output_tokens': 8, 'total_tokens': 26, 'input_token_details': {'cache_read': 0}})

In [ ]:
chain_memoria.invoke(
    {"input": "Qual foi a minha ultima mensagem?"},
    {"configurable": {"session_id": "unused"}},
)

AIMessage(content='Sua última mensagem foi "Olá".\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-0d384d35-aea7-4510-923e-5756a2ca9ef9-0', usage_metadata={'input_tokens': 35, 'output_tokens': 8, 'total_tokens': 43, 'input_token_details': {'cache_read': 0}})

In [ ]:
response = chain_memoria.invoke(
    {"input": "Ah ok, muito obrigado!"},
    {"configurable": {"session_id": "unused"}},
)